In [1]:
from langgraph.graph import StateGraph, END
from typing import TypedDict
from langchain_ollama import OllamaLLM
import sqlite3

# Llama3.2 from your PM2.5 project
llm = OllamaLLM(model="llama3.2:1b")

class AgentState(TypedDict):
    query: str
    expenses: list
    analysis: str

def init_db():
    """Create receipts database with Sri Lankan sample data"""
    conn = sqlite3.connect('receipts.db')
    conn.execute('''CREATE TABLE IF NOT EXISTS expenses
                 (date TEXT, amount REAL, category TEXT, description TEXT)''')
    
    # Sri Lankan LKR sample receipts
    sample_data = [
        ('2026-01-15', 250.5, 'Food', 'Rice from Keells Negombo'),
        ('2026-01-15', 300.0, 'Transport', 'Uber to Colombo'),
        ('2026-01-16', 1500.0, 'Rent', 'Monthly boarding Negombo'),
        ('2026-01-16', 75.0, 'Food', 'Tea from local kadé'),
        ('2026-01-16', 1200.0, 'Shopping', 'Clothes from Odel')
    ]
    
    conn.executemany("INSERT OR IGNORE INTO expenses VALUES (?,?,?,?)", sample_data)
    conn.commit()
    conn.close()
    print("✅ Database ready: 5 LKR receipts loaded!")

# Initialize everything
if __name__ == "__main__":
    init_db()
    print("🚀 FINANCE AGENT SETUP COMPLETE!")
    print("💾 receipts.db created with Sri Lankan expenses")
    print("🤖 Llama3.2 loaded and ready")
    print("\n📋 AGENT STATUS: READY FOR TOOLS")


✅ Database ready: 5 LKR receipts loaded!
🚀 FINANCE AGENT SETUP COMPLETE!
💾 receipts.db created with Sri Lankan expenses
🤖 Llama3.2 loaded and ready

📋 AGENT STATUS: READY FOR TOOLS


In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict
from langchain_ollama import OllamaLLM
import sqlite3
import json

llm = OllamaLLM(model="llama3.2:1b")

class AgentState(TypedDict):
    query: str
    expenses: list
    analysis: str

# TOOL #1: Fetch expenses from database
def get_expenses(state: AgentState) -> AgentState:
    """Agent retrieves all receipts from SQLite"""
    conn = sqlite3.connect('receipts.db')
    cursor = conn.execute("SELECT date, amount, category, description FROM expenses")
    expenses = [{"date": row[0], "amount": row[1], "category": row[2], "desc": row[3]} 
                for row in cursor.fetchall()]
    conn.close()
    
    print(f"💰 TOOL #1: Found {len(expenses)} receipts")
    return {"expenses": expenses}

def init_db():
    """Sri Lankan LKR sample data"""
    conn = sqlite3.connect('receipts.db')
    conn.execute('''CREATE TABLE IF NOT EXISTS expenses
                 (date TEXT, amount REAL, category TEXT, description TEXT)''')
    
    sample_data = [
        ('2026-01-15', 250.5, 'Food', 'Rice from Keells Negombo'),
        ('2026-01-15', 300.0, 'Transport', 'Uber to Colombo'),
        ('2026-01-16', 1500.0, 'Rent', 'Monthly boarding Negombo'),
        ('2026-01-16', 75.0, 'Food', 'Tea from local kadé'),
        ('2026-01-16', 1200.0, 'Shopping', 'Clothes from Odel')
    ]
    
    conn.executemany("INSERT OR IGNORE INTO expenses VALUES (?,?,?,?)", sample_data)
    conn.commit()
    conn.close()

# TEST TOOL #1
if __name__ == "__main__":
    init_db()
    print("🧠 Testing TOOL #1: get_expenses()...")
    
    # Simulate agent state
    test_state = {"query": "show my expenses", "expenses": [], "analysis": ""}
    result = get_expenses(test_state)
    
    print("✅ TOOL #1 SUCCESS!")
    print(f"💰 Retrieved: {len(result['expenses'])} receipts")
    print("📋 First receipt:", result['expenses'][0])
    print("\n🚀 AGENT TOOLS STATUS: 1/3 COMPLETE")


🧠 Testing TOOL #1: get_expenses()...
💰 TOOL #1: Found 76 receipts
✅ TOOL #1 SUCCESS!
💰 Retrieved: 76 receipts
📋 First receipt: {'date': '2026-01-15', 'amount': 250.5, 'category': 'Food', 'desc': 'Rice from Keells Negombo'}

🚀 AGENT TOOLS STATUS: 1/3 COMPLETE


In [3]:
import sqlite3
import plotly.express as px
import json
from langchain_ollama import OllamaLLM
import pandas as pd

llm = OllamaLLM(model="llama3.2:3b")

print("🚀 SIMPLE FINANCE AGENT - 51 RECEIPTS READY!")

# TOOL 1: Get your 51 receipts
def get_expenses():
    conn = sqlite3.connect('receipts.db')
    df = pd.read_sql_query("SELECT * FROM expenses", conn)
    conn.close()
    print(f"💰 Found {len(df)} receipts: Rs.{df['amount'].sum():,.0f} total")
    return df

# TOOL 2: Create pie chart INSTANTLY
def make_chart(df):
    category_totals = df.groupby('category')['amount'].sum()
    fig = px.pie(values=category_totals.values, names=category_totals.index,
                title=f"Your 51 Receipts: Rs.{df['amount'].sum():,.0f}")
    fig.write_html("spending_chart.html")
    print("📊 Chart saved! Open spending_chart.html")
    return category_totals

# TOOL 3: Quick analysis (NO slow LLM)
def quick_analysis(df):
    total = df['amount'].sum()
    food_pct = df[df['category']=='Food']['amount'].sum() / total * 100
    analysis = f"""
    💰 TOTAL SPENDING: Rs.{total:,.0f}
    🍚 FOOD: {food_pct:.1f}% of budget  
    💡 TIP: You're spending {food_pct:.0f}% on food - cook rice & curry at home!
    """
    print("🧠 ANALYSIS COMPLETE!")
    return analysis

# RUN EVERYTHING
if __name__ == "__main__":
    df = get_expenses()
    chart = make_chart(df)
    analysis = quick_analysis(df)
    
    print("\n🎉 FINANCE AGENT COMPLETE!")
    print(analysis)
    print("✅ Open spending_chart.html NOW!")


🚀 SIMPLE FINANCE AGENT - 51 RECEIPTS READY!
💰 Found 76 receipts: Rs.50,307 total
📊 Chart saved! Open spending_chart.html
🧠 ANALYSIS COMPLETE!

🎉 FINANCE AGENT COMPLETE!

    💰 TOTAL SPENDING: Rs.50,307
    🍚 FOOD: 11.7% of budget  
    💡 TIP: You're spending 12% on food - cook rice & curry at home!
    
✅ Open spending_chart.html NOW!
